In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from ase import build
import numpy as np
from lammps import PyLammps, lammps
from mpi4py import MPI

In [3]:
os.chdir('test')

In [4]:
os.getcwd()

'/home/joe/PycharmProjects/Ge_analysis_local/test'

In [5]:
env = os.environ
env['rundir'] = 'run_1000_test'
env['system'] = 'Si'
env['model'] = 'MTP'
env['units'] = 'metal'
env['restart_from'] = 'data'
env['data_file'] = '/home/joe/scripts/lammps/rnd_64.data'
env['pair_style'] = 'mlip {}/mlip.ini'.format(env['rundir'])
env['pair_coeff'] = "* *"
env['p'] = '0'
env['pot'] = 'Si_myDB_liqamocryst_train_24_rb_12.mtp'

In [46]:
os.makedirs(os.path.join(os.environ['rundir'],'NPT'))

ini_str = 'mtp-filename {}\nselect FALSE'.format(env['pot'])
with open(os.path.join(env['rundir'], 'mlip.ini'), 'w') as f:
    f.write(ini_str)

In [65]:
lmp = lammps()
L = PyLammps(ptr=lmp)

LAMMPS output is captured by PyLammps wrapper


In [48]:
L.log(os.path.join(env['rundir'], 'log_npt_{}_{}.dat'.format(env['system'], env['model'])), 'append')

In [49]:
L.units(env['units'])
L.atom_style('atomic')
if env['restart_from'] == 'continuation':
    L.read_restart(env['rundir'], 'restart_npt_{}_{}.*'.format(env['system'], env['model']))
else:
    L.read_data(env['data_file'])

L.mass(1, 28.09)

In [70]:
L.pair_style(env['pair_style'])
L.pair_coeff(env['pair_coeff'])
L.neighbor(2.0, 'bin')
L.neigh_modify('every', 1, 'delay', 0, 'check', 'yes')

Nfreq = 100; Nevery = 10; Nprint = 10; Nrepeat = Nfreq//Nevery; Ndump = 100
# L.variable('Nfreq', 'equal', Nfreq)
# L.variable('Nprint', 'equal', Nprint)
# L.variable('Nevery', 'equal', Nevery)
# L.variable('Nrepeat', 'equal', Nrepeat)
# L.variable('Ndump', 'equal', Ndump)

L.group('silicon', 'type', 1)
L.timestep(0.001)

In [71]:
L.variable('nAtoms equal atoms')
L.fix('removeMomentum', 'all', 'momentum', 1, 'linear', 1, 1, 1)

L.compute('T', 'all', 'temp')
L.fix('TempAve', 'all', 'ave/time', Nevery, Nrepeat, Nfreq, 'c_T')

L.variable('P equal press')
L.fix('PressAve', 'all', 'ave/time', Nevery, Nrepeat, Nfreq, 'v_P')

L.variable('v equal vol')
L.fix('vAve', 'all', 'ave/time', Nevery, Nrepeat, Nfreq, 'v_v')

L.compute('PE', 'all', 'pe', 'pair')
L.variable('PE_atom equal c_PE/v_nAtoms')
L.fix('PEAve_Atom', 'all', 'ave/time', Nevery, Nrepeat, Nfreq, 'v_PE_atom')

L.compute('MSD', 'all', 'msd')

L.thermo_style('custom', 'step', 'cpu',
               'temp', 'f_TempAve',
               'press', 'f_PressAve',
               'f_PEAve_Atom',
               'vol', 'f_vAve',
               'c_MSD[4]', 'fmax')

L.thermo(Nfreq)
L.thermo_modify('flush', 'yes')

L.dump('trj_movie all cfg {} {}/NPT/dump_npt_{}_{}.*.cfg mass type xs ys zs'.format(
        Nfreq, env['rundir'], env['system'], env['model']))
L.dump_modify('trj_movie element {}'.format(env['system']))

L.restart(Ndump, '{}/restart_npt_{}_{}.*'.format(env['rundir'], env['system'], env['model']))

In [52]:
# env['stages'] = 'melt liquid super_liquid quench amo'
env['stages'] = 'melt liquid super_liquid'
# env['steps'] = '2e4 1e5 1e5 1e5 2e4'
env['steps'] = '3e2 3e2 3e2 3e2 3e2'
# env['temps'] = '2500 2500  1800 1800  1500 1500  1500 300  300 300'
env['temps'] = '2500 2500  1800 1800'
stages = env['stages'].split()
steps = [int(float(i)) for i in env['steps'].split()]
temps = [float(i) for i in env['temps'].split()]
p_bar = [float(i)*1e4 for i in env['p'].split()]
if len(p_bar) != len(temps):
    print('Global constant pressure set')
    p_bar = [p_bar[0] for i in range(len(temps))]

for i in range(len(stages)):
    L.variable(stages[i], 'equal', sum(steps[:i+1]))

Global constant pressure set


In [53]:
if L.eval('step') == 0 and steps[0] > 0:
    np.random.seed()
    rand = np.random.randint(0, high=np.iinfo(np.int32).max, dtype=np.uint32)
    print(rand)
    L.velocity('all', 'create', temps[0], rand)

992114514


In [54]:
L.run(0)
step = 0

for i in range(len(stages)):
    L.fix('integrate all npt',
          'temp', temps[2*i], temps[2*i+1], 0.1,
          'iso', p_bar[2*i], p_bar[2*i+1], 1.0,
          'nreset', 1000)
    # if step < steps[i]:
        # L.run(sum(steps[:i+1]), 'upto', 'start', sum(steps[:i]), 'stop', sum(steps[:i+1]), 'pre', 'no', 'post', 'no')
    L.run(sum(steps[:i+1]), 'upto')
    step += steps[i]
    L.unfix('integrate')
    L.write_data('{}/post_{}_data_npt_{}_{}'.format(env['rundir'], stages[i], env['system'], env['model']))

In [72]:
L.run(10)

['Unused restart file global fix info:',
 '  fix style: npt, fix ID: integrate',
 'All restart file peratom fix info was re-assigned',
 'Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 7',
 '  ghost atom cutoff = 7',
 '  binsize = 3.5, bins = 3 3 3',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair mlip, perpetual',
 '      attributes: full, newton on',
 '      pair build: full/bin/atomonly',
 '      stencil: full/bin/3d',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : metal',
 '  Current step  : 600',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 4.709 | 4.709 | 4.709 Mbytes',
 'Step CPU Temp f_TempAve Press f_PressAve f_PEAve_Atom Volume f_vAve c_MSD[4] Fmax ',
 '     600            0    1804.6374            0    -3064.086            0            0    1149.4449            0    10.916384    2.

In [64]:
L.eval('step')

610.0

In [67]:
L.read_restart(os.path.join(env['rundir'], 'restart_npt_{}_{}.*'.format(env['system'], env['model'])))

['Reading restart file ...',
 '  restart file = 29 Oct 2020, LAMMPS = 29 Oct 2020',
 '  restoring atom style atomic from restart',
 '  orthogonal box = (0.25384204 0.25384204 0.25384204) to (10.729051 10.729051 10.729051)',
 '  1 by 1 by 1 MPI processor grid',
 '  pair style mlip stores no restart info',
 '  64 atoms',
 '  read_restart CPU = 0.003 seconds']

In [73]:
L.eval('step')


610.0